La idea es proponer que el número de casos confirmados  $C(t)$ es de la forma
$$
C_t=\exp(f(t)+\delta_t)
$$
con $f(t)$ una función tipo logística:
$$
f(t)=\frac{h_1-h_0}{1+\exp( (t_0-t)/a )} + h_0
$$
y $\delta(t)$ una fluctuación estadística con una amplitud constante, de manera que $C_t$ tenga una fluctuación estadística relativa constante. 
Para encontrar los parámetros, ajustaremos entonces $\log C_t$ con la curva $f(t)$.


Si $t_0$ es negativo y grande en valor absoluto, de manera que $-t_0\gg a$, la curva se puede aproximar por
$$
f(t)= h_1 + (h_0-h_1)e^{-|t_0|/a}\exp(-t/a)
$$
que representa una relajación exponencial en todo el intervalo.

In [1]:
import scipy.optimize as scopt
%matplotlib inline
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rnd
import pandas as pd
import json

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

fits = {} 


data = pd.read_json("../Datos/data.json")
nodes = {}
for d in data:
    nodes[d] = {}
    for entry in  data[d].keys():
        nodes[d][entry] =  data[d][entry]


In [5]:

# Función a ajustar
def  logistica(t,t0,a,h0,h1):
    if t is np.ndarray:
        return np.array([logistica(tc,t0,a,h1,h0) for tc in t])
    else:
        return (h1-h0)*(1-np.exp(-t/a))/(1.+np.exp((t0-t)/a)) + h0

    
def  logistica2(t,t0,a,h0,h1,ln0):
    if t is np.ndarray:
        return np.array([logistica2(tc,t0,a,h1,h0) for tc in t])
    else:
        ex1 = np.exp((t-t0)/a)
        ex2 = np.exp(-t0/a)
        t1 = h0 * (t-t0) + ln0
        t2 =  a * np.log((ex1+1)/(ex2+1)) * (1-ex2) + t * ex2
        return t1 + (h1-h0)*t2
    
def ajustar(pais,curva, ajuste="log", maxday=None,threeshold = 100):
    dictajuste = {}    
    data = np.array(nodes[pais]["curve_"+curva])
    if ajuste == "log":
        lndata = np.log(data)
        lndata = lndata[lndata >= np.log(threeshold)]
    else:
        lndata = data[data > 0]
        lndata = lndata[lndata >= threeshold]
        
    
    lndata = lndata
    startday = len(data) - len(lndata)
    days = np.array(range(len(lndata)))
    if maxday is not None:
        maxday = min(maxday,len(lndata))
        days = days[:maxday]
        lndata = lndata[:maxday]
    if len(lndata)<8:
        dictajuste["t_0"] = np.nan
        dictajuste["\\tau"] = np.nan
        dictajuste["h_0"] = np.nan
        dictajuste["h_1"] = np.nan
        dictajuste["ln_0"] = np.nan
    try:        
        timescale = max(days)
        scale =  max(lndata)
        days = (days + startday) / timescale
        lndata = lndata/scale
        if ajuste == "log":
            fitl,covl = scopt.curve_fit(lambda x,a,b: a*x+b,days,lndata,method="dogbox")
            p0 = ( .5 *(days[-1]+days[0]),1,fitl[0],fitl[0],lndata[0])
            fit, cov = scopt.curve_fit(logistica2,days,lndata,method="dogbox")
            dictajuste["ln_0"] =  fit[4]*scale
            dictajuste["h_0"] = fit[2]*scale/timescale
            dictajuste["h_1"] = fit[3]*scale/timescale
        else:
            try:
                fit, cov = scopt.curve_fit(logistica,days,lndata)
            except OptimizeWarning:
                print(fit)
            
                
            dictajuste["h_0"] = fit[2]*scale
            dictajuste["h_1"] = fit[3]*scale
            
        dictajuste["t_0"] = timescale * fit[0]
        dictajuste["\\tau"] = fit[1] * timescale
        
        
    except:
        dictajuste["t_0"] = np.nan
        dictajuste["\\tau"] = np.nan
        dictajuste["h_0"] = np.nan
        dictajuste["h_1"] = np.nan 
        dictajuste["ln_0"] = np.nan
    return dictajuste


def mostrar_ajuste(regions, curva,tipo,maxday=None,threeshold=1):
    if curva == "Confirmados":
        curva = "confirmed"
    if curva == "Recuperados":
        curva = "recovered"
    if curva == "Muertos":
        curva = "deaths"
    if type(regions) is not list and type(regions) is not tuple :
        regions = [regions]
    
    if tipo == "ambos":
        fig, axs = plt.subplots(1,3,figsize=(15,4))
    else:
        fig, axs = plt.subplots(1,2,figsize=(10,4))
        
    for region in regions:
        axs[0].set_xlim(0,2)
        axs[0].set_ylim(0,2)
        axs[0].set_axis_off()
        ax = axs[1]
        axs[0].text(.6,1.9,region)
        if tipo in ["ambos", "normal"]:
            data = np.array(nodes[region]["curve_"+curva])
            days = np.array(range(len(data)))
            ax.scatter(days,data,label=region+" "+curva)
            try:
                fit = ajustar(region,curva,"normal",maxday = maxday,threeshold=np.exp(threeshold))
                ax.plot(days,logistica(days,fit["t_0"],fit["\\tau"],fit["h_0"],fit["h_1"]))
                ax.set_title("Ajuste logistico de C_t")
                axs[0].text(0.2,1.6,"Param Logistica")
                for i,k in enumerate(fit):
                    axs[0].text(0.2,1.4-.2*i,"$"+k +"="+str(round(fit[k],2))+"$"  )
                axs[0].text(0.2,.5,"Prediccion al\n 1 de mayo:"+str(round(logistica(100,
                                                                           fit["t_0"],
                                                                           fit["\\tau"],
                                                                           fit["h_0"],
                                                                           fit["h_1"]))   )  )
            except:
                pass
                ax.legend()
        if tipo == "ambos":
            ax = axs[2]
        if tipo in ["ambos", "log"]:
            data = np.array(nodes[region]["curve_"+curva])
            days = np.array(range(len(data)))
            ax.scatter(days,np.log(data),label=region+" "+curva)
            try:
                fit = ajustar(region,curva,"log",maxday = maxday,threeshold=np.exp(threeshold))
                ax.plot(days,logistica2(days,fit["t_0"],fit["\\tau"],fit["h_0"],fit["h_1"],fit["ln_0"]))
                axs[0].text(1,1.6,"Param $\\int logistic(t)dt$")
                for i,k in enumerate(fit):
                    axs[0].text(1,1.4-.2*i,"$" + k +"="+str(round(fit[k],2))+"$")        
                axs[0].text(1.1,.3,"Prediccion al\n 1 de mayo:"+str(round(np.exp(logistica2(100,
                                                                           fit["t_0"],
                                                                           fit["\\tau"],
                                                                           fit["h_0"],
                                                                           fit["h_1"],
                                                                           fit["ln_0"])))   )  )

            except:
                pass
            ax.set_title("Ajuste logistico de log(C_t)")
            ax.legend()
        
    plt.show()
        
        
        
    
        


In [6]:

regionesagraficar = widgets.SelectMultiple(options=nodes.keys(),
    description='País/Región:',
    value=["Argentina"],
    disabled=False)

curva = widgets.Dropdown(options=['Confirmados', 'Recuperados', 'Muertos'],
    description='Curva:',
    ensure_option=True,
    disabled=False)


tipo=widgets.Dropdown(options=["ambos",'normal', 'log'],
    description='Tipo de ajuste:',
    ensure_option=True,
    disabled=False)

maxday=widgets.IntSlider(100,0,100,
    description='Número de días a ajustar',
    ensure_option=True,
    disabled=False)

threeshold=widgets.IntSlider(2,0,12,
    description='Descartar dias antes de tener e^()casos',
    ensure_option=True,
    disabled=False)



interact=widgets.interact(mostrar_ajuste,regions=regionesagraficar,curva=curva,tipo=tipo,
                          maxday=maxday,threeshold=threeshold)


#ajustar("Argentina","confirmed","log",10)

interactive(children=(SelectMultiple(description='País/Región:', index=(0,), options=('Argentina', 'Bolivia', …

In [7]:
np.log(2)/.3

2.3104906018664844